# 第 3 章：近似理论（Approximation Theory for Quantization & Cheap Ops）

近似理论回答的问题是：

> “在允许一定误差的前提下，如何用更简单的函数 / 更粗的精度来逼近原函数？”

在推理加速中主要应用于：

- 量化误差建模
- 激活函数近似（GELU/SiLU/tanh 等）
- 用多项式 / 分段线性函数替代复杂算子
- approximate computing（例如用 cheap ops 替换 expensive ops）

---

## 3.1 均匀量化的误差模型

考虑步长为 $\Delta$ 的均匀量化：

$$
\hat{w} = Q(w) = \Delta \cdot \text{round}\Big(\frac{w}{\Delta}\Big)
$$


定义量化误差：

$$
e = w - \hat{w}
$$


在很多假设下（信号在每个量化区间内分布比较“均匀”），可以近似认为：

- $e$ 在 $[-\Delta/2, \Delta/2]$ 上均匀分布
- 即：$e \sim \mathcal{U}(-\Delta/2, \Delta/2)$

于是：

$$
\mathbb{E}[e] \approx 0, \quad
\mathbb{E}[e^2] = \text{Var}(e) \approx \frac{\Delta^2}{12}
$$


**【工程意义】** 
- 量化噪声方差与步长平方成正比。 
- 量化步长 $\Delta$ 越大 → 误差方差越大
- 对某层使用更低 bit（更少级别）时，可以估算噪声能量：$\sigma_e^2 \approx \Delta^2/12$

这为“按层动态选择 bit 宽”提供了理论依据：

- 对更敏感的层使用小 $\Delta$（更多 bit）
- 对不敏感的层使用大 $\Delta$（更少 bit）

---

## 3.2 激活函数的近似：以 GELU 为例
### 3.2.1 近似数学原理
GELU（Gaussian Error Linear Unit） 定义：

$$
\text{GELU}(x) = x \cdot \Phi(x)
$$


其中 $\Phi(x)$ 是标准正态分布的 CDF：

$$
\Phi(x) = \frac{1}{\sqrt{2\pi}} \int_{-\infty}^x e^{-t^2/2} \, dt
$$


直接计算 CDF 非常昂贵，因此常用近似：

$$
\text{GELU}(x) \approx 0.5 x \left(1 + \tanh\left[\sqrt{\frac{2}{\pi}}(x + 0.044715 x^3)\right]\right)
$$

```python
def gelu_approx(x):
    return 0.5 * x * (1 + torch.tanh(
        math.sqrt(2 / math.pi) * (x + 0.044715 * x ** 3)
    ))

```

进一步，可以将 tanh 又近似为一个多项式或分段线性函数，使整体计算只包含：

- 加法
- 乘法
- 少量表查

**【近似理论视角】**  

- 在一个有限区间 $[a,b]$ 上，用多项式 $P_n(x)$ 逼近一个平滑函数 $f(x)$ 是可行的：

$$
\|f - P_n\|_{\infty, [a,b]} \to 0 \quad \text{(当 n → ∞ 时)}
$$


- 对应到工程中，就是选取一个多项式阶数 n，使得：
  - 逼近误差足够小（精度要求）
  - 计算代价足够低（乘法/加法次数）

### 3.2.2 为什么用GELU而不是RELU
#### 3.2.2.1 数学性质：ReLU 是分段线性，GELU 是平滑可导

**ReLU：**

$$
\mathrm{ReLU}(x) = \max(0, x)
$$

- 在 $x=0$ 不可导  
- 左侧完全为 0（死区）  
- 梯度是 0 或 1（硬切断）

**GELU：**

$$
\mathrm{GELU}(x) = x \, \Phi(x)
$$

其中 $\Phi(x)$ 是标准正态分布 CDF。

- 全程连续可导  
- 梯度平滑  
- 适用于深层网络和残差结构

➡ **Transformer 需要平滑的非线性，而不是 ReLU 这种“硬切断”。**

---

#### 3.2.2.2 Transformer 的深层结构需要更稳定的激活梯度

Transformer 中有大量：

- Residual connections  
- LayerNorm  
- Attention（高度耦合）  
- 深 MLP  
- 数十甚至数百层深度

ReLU 容易造成：

- 梯度突然跳变（0 ↔ 1）  
- 特征被切死（dead neurons）  
- 残差路径中断  
- 深层梯度碎裂

GELU：

- 保持梯度连续  
- 让深层网络梯度传播更稳定  
- 大幅提升收敛性能

➡ **深度 Transformer 结构用 GELU 的训练稳定性显著优于 ReLU。**

---

#### 3.2.2.3 GELU 能学习“输入重要性”，ReLU 不能

ReLU 的行为非常粗糙：

- $x > 0$：全部通过  
- $x \le 0$：全部切断  

不关心“小负数 vs 大负数”的区别。

GELU 本质上是：

$$
\text{“按概率通过”} = x \cdot \Phi(x)
$$

直觉：

- 小正数 → 部分通过  
- 小负数 → 部分抑制  
- 大正数 → 完全通过  
- 大负数 → 完全抑制  

➡ **GELU 有 soft gating 机制，比 ReLU 的硬切断强得多。**

这点对表示能力和模型容量影响巨大。

---

#### 3.2.2.4 实证：所有 SOTA 大模型都不用 ReLU

| 模型 | 激活函数 |
|------|----------|
| BERT | GELU |
| GPT-2 / GPT-3 / GPT-4 | GELU / SiLU |
| PaLM | GELU |
| T5 | GELU |
| ViT | GELU |
| LLaMA / Qwen | SiLU（Swish） |
| Stable Diffusion / DiT | SiLU |

ReLU 基本从现代 Transformer 中消失。

➡ **行业共识：Transformer 必须用 GELU/SiLU，而不是 ReLU。**

---

#### 3.2.2.5 量化视角：ReLU 输出无界，GELU 输出更“高斯化”

**ReLU 输出范围：**

$$
[0, +\infty)
$$

这会导致：

- 激活值出现长尾  
- int8/int4 量化 scale 不稳定  
- 大值压缩小值（降低有效 bit 精度）  
- 容易饱和（尤其在 int4 中）

**GELU 输出分布：**

- 更接近高斯  
- 有自然的抑制  
- 输出值范围更可控  
- 更利于量化（更好的对称性 + 更小的 tail）

➡ **GELU 比 ReLU 更适合 quantization-aware inference。**

这对你做 Runtime Inference Acceleration 非常关键。

---

#### 3.2.2.6 工程层面：GELU 稍贵，但对总推理时间影响极小

ReLU 是极快（一个 `max` 操作）。

但是 Transformer 的开销主要在：

- GEMM / MatMul  
- Attention  
- MLP Projection

激活函数通常 < 1% 的计算成本。

GELU 的额外成本可以通过：

- fused kernels  
- polynomial approximation  
- LUT approximation  
- fast GELU (tanh-based)  

来大幅减少。

➡ **GELU 的计算成本远不如它在性能上的好处重要。**

---

#### 3.2.2.7 最终总结
| 对比点               | GELU      | ReLU              |
| ----------------- | --------- | ----------------- |
| 平滑性               | ✔ 连续可导    | ✘ 在 0 不可导         |
| 梯度传播              | ✔ 平滑且稳定   | ✘ 梯度跳变 0→1        |
| 深层网络（Transformer） | ✔ 强稳定性    | ✘ 易出现梯度碎裂         |
| 表达能力              | ✔ “概率性保留” | ✘ “硬切断”           |
| 量化友好性             | ✔ 输出分布好   | ✘ 长尾影响 scale      |
| 大模型最佳实践           | ✔ 默认选择    | ✘ 不用于 Transformer |
| kernel 代价         | 略贵但可接受    | 超快                |

Transformer 是深、宽、残差耦合的结构，需要平滑、可导、稳定梯度的激活函数。  
GELU 在训练稳定性、梯度流动、表达能力和量化友好性上全面优于 ReLU，  
因此现代大模型几乎统一选择 GELU/SiLU，而不是 ReLU。



## 3.3 多项式近似的构造（最小二乘 + Chebyshev）
### 3.3.1 数学原理
给定函数 $f(x)$，希望在区间 $[a,b]$ 上用多项式：

$$
P_n(x) = \sum_{k=0}^n c_k x^k
$$


来逼近它。

常见方法：

1. **最小二乘拟合**（离散点）：
   - 在区间内采样若干点 $x_i$，最小化：

$$
\sum_i (f(x_i) - P_n(x_i))^2
$$


   - 这会导致一个线性最小二乘问题求 $\{c_k\}$

2. **Chebyshev 逼近**（均匀最大误差最小）：
   - 用 Chebyshev 多项式作为基函数
   - 在理论上能提供更好的最大误差界
使用 Chebyshev 多项式 $T_k(x)$,
$$P_n(x)=\sum_{k=0}^{n} a_k T_k(x)$$

**【对推理工程师的要求】**  
- 不需要会手推 Chebyshev 多项式
- 需要知道：
  - 激活函数/特殊函数的近似来自“多项式拟合”或“有理函数拟合”
  - 阶数越高、基函数越复杂 → 精度越好，但计算也越贵

### 3.3.2 应用场景：在推理加速中用 Chebyshev 多项式逼近激活函数
🧩 背景：为什么推理需要逼近激活函数？

在部署（尤其是边缘端、车载 NPU、DSP、低功耗 MCU）中：

- erf、exp、tanh 这些函数很贵

- NPU 没有原生指令（尤其是 exp 和 erf）

- 高精度实现需要多次乘加、查表、迭代，太慢

- kernel 想达到 1-cycle per op，就必须用简单算子（mul/add）

于是工程上最典型的做法是：

用一个低阶的多项式或分段多项式来逼近激活函数
—— 尽可能便宜，又保证误差在可接受范围内。

但是不希望：

- 某些点误差很大

- 某个 x 区域拟合很差

- 影响后续 layernorm、attention、logits

- 导致量化误差扩大

- 所以要找 全区间最大误差最小 的逼近。

这就是 Chebyshev 最小最大（minimax）逼近存在的理由。
⭐ 例子：在区间 [-3, 3] 上用 3 阶 Chebyshev 多项式逼近 tanh(x)
在 GM 的车载视觉模型里，有一个 SiLU 或 tanh 激活函数。
NPU 上没有 tanh 指令，但有：

- 加法

- 乘法

- FMA（fused multiply-add）

- bitshift（整数右移）

小容量 lookup table

目标：
写一个只用 mul/add 的自定义 kernel 来跑 tanh。

但模型精度敏感，如果近似不好，会影响：

- self-attention 输入分布

- layernorm 稳定性

- logits 分布

- 量化 calibration

- 最终 AP / accuracy

你不能随便拟合一个多项式，需要控制“最坏情况误差”。

✔ 使用 Chebyshev 最小最大误差近似
目标：

在区间 [-3,3] 上找一个 3 次多项式， $P_3(x) = a_0 + a_1 x + a_2 x^2 + a_3 x^3$ 使最大误差 $\max_{x\in [-3,3]} |P_3(x) - \tanh(x)|$ 最小
Chebyshev 理论告诉我们：

- 最优逼近一定是以 Chebyshev 多项式为基

- 系数可通过 Remez 交换算法得到

- 得到的函数具有 等振性（equioscillation）：

- 最大误差在区间内的若干点上完全相等


🌟 结果是什么？（工程可用版本）
逼近 tanh(x) 的 Chebyshev minimax 多项式（3 阶）可能是： 
$$P_3^{*}(x) = 0.814 x - 0.066 x^3$$ 而且

- 使用 1 个乘方 + 2 个乘法 + 1 个减法

- 在 NPU/DSP 上可用 fused ops

现在kernel就有
```python
// 超快 tanh 近似，基于 Chebyshev minimax
float fast_tanh(float x) {
    float x3 = x * x * x;
    return 0.814 * x - 0.066 * x3;
}
```

🌟 Note: 这个和通过泰勒展开得到 然后截断的3阶
$\tanh(x) = x - \frac{x^3}{3} + \frac{2x^5}{15} - \cdots$
$$T_3(x) = x - \frac13 x^3$$
是不一样的。 它的性质是：在 𝑥=0 附近误差最小，但远离 0 误差急剧增大； 最小最大误差（minimax）意义下最佳的三次多项式，泰勒展开只有在靠近参考点出才有好的近似价值

⭐ 主流加速库使用
CoreML 的 fast GELU

TensorRT 的 GELU

ARM CMSIS 的 tanh/sigmoid

TFLite 的 exp/log

EdgeTPU 的 activation poly

📌 三个方法的差异：
| 方法                             | 目标                | 结果                    | 优点         | 缺点              |
| ------------------------------ | ----------------- | --------------------- | ---------- | --------------- |
| **泰勒近似**                       | 在中心点（(x=0)）附近误差最小 | 例：(x - \frac13 x^3)   | 公式简单       | 区间外误差巨大，不适合推理部署 |
| **最小二乘近似**                     | 平均误差最小            | 系数介于两者之间              | L2-optimal | 最坏点误差可能很大       |
| **Chebyshev / Remez 最小最大误差近似** | 最坏点误差最小           | 例：(0.814x - 0.066x^3) | 工程最优、推理最友好 | 计算复杂，需要 Remez   |



In [ ]:
 # Remez 最小最大误差近似实现


## 3.4 Approximate Computing：用“更便宜”的算子替代

典型思路：

- 用 shift 替代乘法（当系数接近 $2^k$ 时）
- 用 LUT（查表）替代复杂函数（如 exp / log / erf）
- 用 piecewise linear 替代 smooth 非线性（如 ReLU6, HardSwish 等）

这都可以看作是：

> 在允许一定误差的前提下，用“更便宜的算子组合”近似原函数。

常见 cheap ops：

- 加法（add）

- 乘法（mul）

- 右移（logical / arithmetic shift）

- max / min

- LUT + 插值

- 分段线性函数（PWL）


典型替代策略：

- 用 shift 近似乘法 w⋅x≈x≪k

- 用分段线性逼近 exp/log
 - 当 $x \approx 0$： $e^{x} \approx 1 + x$（泰勒一阶）
 - 粗略近似（例如在硬件或 LUT 中）： $e^{x} \approx 2^{\mathrm{round}(x / \ln 2)}$

- HardSwish 作为 Swish 近似
$$\mathrm{HS}(x) = x \cdot \frac{\mathrm{ReLU6}(x + 3)}{6}$$

**【与你的工作关系】**  

- 在设计自定义 kernel 时，你可以：
  - 通过数学近似把复杂算子分解成“mul+add+max+shift+table lookup”
  - 利用硬件对这些基本算子的高速支持
- 在评估近似时，你需要有：
  - 定性直觉：误差对下游的影响（例如激活输出范围变窄/变宽）
  - 简单定量工具：最大误差、L2 误差等

---

## 3.5 小结：统一视角

所有这些近似方法都可以统一写成：

$$
\hat{f}
= \arg\min_{g \in \mathcal{H}_{\text{cheap}}} \;\|f - g\|_{\Omega}.
$$

其中：
- $\mathcal{H}_{\text{cheap}}$：由 cheap ops 构成的函数族
- $\Omega$：定义误差的度量（如 $L_2$、$L_\infty$、KL 等）

这说明：
- 量化：对数值范围的分段常数逼近
- 激活近似：对激活函数的多项式/分段逼近
- LUT + 插值：piecewise polynomial 逼近
- shift-based kernel：对乘法算子的近似

本质：在更低复杂度的函数族中寻找最优逼近。
